In [ ]:
# use below command in pip if widget extensions aren't displaying
# jupyter nbextension enable --py widgetsnbextension

In [ ]:
import datetime
import json
import numpy as np
import pandas as pd
import requests

from io import StringIO
from urllib.request import urlopen
from scipy import stats
from scipy.stats import kurtosis
from scipy.stats import skew
from statsmodels.robust.scale import mad

import ipywidgets as widgets
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from IPython.display import display
from ipywidgets import Layout

In [ ]:
print('Retrieving data from NY Times GIT...'\
     'Note that the data is usually two days behind.')
url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
resp=requests.get(url).text
df=pd.read_csv(StringIO(resp))

#need to add save to file in case the service goes down

In [ ]:
print('Uploading State CSV file...')
df_csv=pd.read_csv('state_locs.csv')

## Perform data cleanup

In [ ]:
# determine how many rows are in the table
rowcount = df.shape[0]

# determine how many rows have null FIPS 
null_fips_rowcount = df.loc[df['fips'].isnull()].shape[0]

# calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

print("There were "+str(null_fips_rowcount)+" records with null FIPS values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

In [ ]:
# create a new dataframe without null FIPS records
df = df.loc[df['fips'].notnull()].reset_index(drop=True)

# convert FIPS data to int to remove decimal point
df = df.astype({'fips': 'int64'})
# convert back to str for processing
df = df.astype({'fips': 'str'})

# check how many records have a FIPS value with four characters
trunc_df = df.loc[df['fips'].str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / df.shape[0])*100

# use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

In [ ]:
# pad zero to fips with len 4
conditions = [df['fips'].str.len()==4]
choices = ['0'+ df['fips']]

df['fips'] = np.select(conditions, choices, default=df['fips'])

In [ ]:
df.tail(10)

In [ ]:
# how far back to limit the data
DAYS_BACK= 30

# Common Functions

In [ ]:
def date_limit(df, days_back=14) -> pd.DataFrame():
    """
    Common date filter for dfs
    """
    
    df = df.copy(deep=True)
    today = pd.to_datetime('today')
    df.loc[:,'days_since'] = (today-df.index).days
    
    return df.loc[df['days_since']<=days_back]
    

def determine_daily_counts(group) -> pd.DataFrame():
    """
    Calculate daily cases/death counts
    cases are cumulative, so using the diff method 
    to determine daily counts
    """    
    group.loc[:,'daily_case_count'] = group['cases']
    group.loc[:,'daily_case_count'] = group['cases'].diff().fillna(group['cases']).astype('int32')
    group.loc[:,'daily_death_count'] = group['deaths'].diff().fillna(0).astype('int32')
    return group


def determine_est_of_location(df, col):
    """
    Prints common estimate of location 
    information for data
    """
    print(f'\nMax: {df[col].describe()[7]}')
    print(f'Min: {df[col].describe()[3]}')
    print(f'Mean: {round(df[col].describe()[1],2)}')
    print(f'Trimmed Mean: {round(stats.trim_mean(df[col], proportiontocut=0.1),2)}')
    print(f'Median: {df[col].median()}')


def determine_skewness(df, col):
    """
    Determines skewness of data
    """
    skew_val = skew(df[col])
    kurtosis_val = kurtosis(df[col])
    
    print(f'skewness value of: {round(skew_val,2)}')
    # if the index is between -1 and 1, then the distribution is symmetric. 
    if -1 <= skew_val <= 1:
        print('data is symmetric')
    # if the index is no more than -1 then it is skewed to the left 
    if skew_val <= -1:
        print('data is left skewed')
        
    # if it is at least 1, then it is skewed to the right
    if skew_val >=1:
        print('data is right skewed')
    
    print(f'\nkurtosis value of: {round(kurtosis_val,2)}')
    # Leptokurtic (Kurtosis > 3): Distribution is longer, tails are fatter. data are heavy-tailed or profusion of outliers
    if kurtosis_val > 3:
        print('distribution is longer and data are heavy-tailed/profusion of outliers')
    
    # Platykurtic: (Kurtosis < 3): Distribution is shorter, tails are thinner than the normal distribution. 
    if kurtosis_val < 3:
        print('distribution is shorter, tails are thinner than normal distribution')
    # data are light-tailed or lack of outliers.


def determine_est_of_variability(df, col):
    """
    Prints common estimate of variability 
    information for data
    """
    print(f'\nSTD: {round(np.std(df[col]),2)}') # population STD
    temp = df.describe(include=[np.number],percentiles=[.10,.90]).T
    tstd = stats.tstd(df[col],(temp['10%'].tolist()[0],temp['90%'].tolist()[0]))
    print(f'Trimmed STD: {round(tstd,2)}')
    print(f'IQR: {df[col].quantile(0.75) - df[col].quantile(0.25)}')
    print(f'Mean absolute deviation: {round(df[col].mad(),2)}')
    print(f'Median absolute deviation: {round(mad(df[col]),2)}')
    

def fix_negative_values(df, col1, col2) -> np.ndarray:
    """
    Finds negative values and 
    sets to calculated value
    """
    condictions = [df[col1] < 0]
    choices = [df[col2]]
    return np.select(condictions, choices, default=df[col1]).astype('int32')


def graph_data(df, column, chart_type, chart_name='data graphed', chart_title='title',color='rgb(26, 118, 255)'):
    """
    Main graph function
    """
    df = df.copy()
    
    # caluculate rolling mean for every 4 days. This is not cumlative.
    df.loc[:,'rolling_mean'] = df[column].rolling(window=4).mean()
    df.loc[:,'rolling_mean'].fillna(0, inplace=True)
    
    # only going to display past x days
    df = date_limit(df,days_back=DAYS_BACK)
    
    if chart_type == 'line':
        # graph daily cases and mean as line chart
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df[column],
                        mode='lines+markers',
                        line=dict(color=f'{color}', width=2),
                        name=f'{chart_name}'))
        fig.update_layout(title=chart_title)

        fig.add_trace(go.Scatter(x=df.index, y=df['rolling_mean'], name='Rolling Mean',
                                 line = dict(color=f'{color}', width=2, dash='dash')))

        fig.update_xaxes(rangeslider_visible=True, title='Date Slider')
        fig.show()
    
    elif chart_type == 'bar':
        # graph weekday daily cases count as bar graph
        df['dtg'] = pd.to_datetime(df['date'])
        weekday = df['dtg'].dt.day_name()
        weekday_daily_sum = df.groupby(weekday)[column].sum().to_frame()
        
        # bar chart name
        state_name = df.state.unique()[0]
        
        # sort weekday df
        sorter = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
        sorterIndex = dict(zip(sorter,range(len(sorter))))
        weekday_daily_sum['day_id'] = weekday_daily_sum.index
        weekday_daily_sum['day_id'] = weekday_daily_sum['day_id'].map(sorterIndex)
        weekday_daily_sum.sort_values('day_id', inplace=True)
        
        # generate weekd day bar graph
        fig = go.Figure([go.Bar(x=weekday_daily_sum.index, y=weekday_daily_sum[column],
                        text=weekday_daily_sum[column],textposition='auto',marker_color=color)])
        fig.update_layout(title=f'{state_name}')
        fig.show()
    
    elif chart_type == 'violin':
        fig = px.violin(df, y=column, points='all', height=600)
        fig.show()
        
    elif chart_type == 'histogram':
        fig = px.histogram(df, x='daily_case_count', histnorm='percent', marginal='rug')
        fig.show()

        
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique


def get_county_geom(df, counties) -> dict:
    """
    Returns county geoms for selected
    state in df
    """
    
    fips_list = df.fips.unique()
    county_geoms = {"type": "FeatureCollection", "features":[]}
    
    for rec in counties['features']:
        if rec['id'] in fips_list:
            county_geoms['features'].append(rec)
    
    return county_geoms


def get_county_lat_lon(geoms) -> float:
    """
    Returns mean lon, lat coords 
    of geoms 
    """
    coords = []
    lons, lats = [], []
    for rec in geoms['features']:
        try:
            lons.append(np.mean(np.asarray(rec['geometry']['coordinates'],dtype=object), axis=(0,1))[0])
            lats.append(np.mean(np.asarray(rec['geometry']['coordinates'],dtype=object), axis=(0,1))[1])
        except(IndexError,TypeError) as err:
            continue
    return round(np.mean(lons),2), round(np.mean(lats),2)


def map_data(df, size_column, title, map_type, lat='lat', lon='lon'):
    """
    Main mapping function
    """
    if map_type == 'scatter':
        df['dtg'] = df.index.astype(str)
        fig = px.scatter_geo(df, locationmode = 'USA-states', lat=lat, lon=lon,
                         hover_name="state", size=size_column, animation_frame="dtg",)
        fig.update_layout(
            title_text = title,
            showlegend = True,
            geo = dict(
                scope = 'usa',
                landcolor = 'rgb(217, 217, 217)',))
        fig.show()
    elif map_type == 'choropleth':
        # get counties geom
        with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
            counties = json.load(response)
            
            with open("counties_geom.json", "w") as f:
                json.dump(counties,f,indent=4,sort_keys=True)
            
            # filter counties dict to state specific per df
            state_counties = get_county_geom(df, counties)
            
            # get mean lon/lat of counties geom
            lon, lat = get_county_lat_lon(state_counties)
        
        # blues
        fig = px.choropleth_mapbox(df, geojson=state_counties, locations=df['fips'], color=size_column,
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=5, center = {"lat": lat, "lon": lon},hover_data=['county', 'total_cnt'],
                           opacity=0.5,
                           labels={'county':'County', 'daily_case_count':'Cases'},
#                            animation_frame = 'date',
                                   title=title
                          )
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.show()
        

# Perform Data Wrangling 

In [ ]:
# create states only no county df
states_df = df.groupby(['state', 'date']).sum().reset_index()
states_df.loc[:,'dtg'] = states_df['date']
states_df.set_index(pd.DatetimeIndex(states_df['date']), inplace=True)
states_df = states_df[['state', 'cases', 'deaths', 'date']]

# create group obj to perform specific state operations
states_df2 = states_df.groupby('state').apply(determine_daily_counts)

# find negative daily counts
states_df2.loc[:,'negative_cases_daily'] = states_df2['daily_case_count'].apply(lambda x: x < 0)
states_df2.loc[:,'negative_deaths_daily'] = states_df2['daily_death_count'].apply(lambda x: x < 0)

# check how many records
negative_cnts_df = states_df2.loc[(states_df2['negative_cases_daily']== True) | (states_df2['negative_deaths_daily']== True)]
negative_cnts_per = (negative_cnts_df.shape[0] / states_df2.shape[0])*100
print(f'{round(negative_cnts_per, 2)}% of the data ({negative_cnts_df.shape[0]} rows out of {states_df2.shape[0]}) have negative counts')

In [ ]:
# see records with bad values
# states_df2.loc[(states_df2['negative_cases_daily']== True) | (states_df2['negative_deaths_daily']== True)]

In [ ]:
# find records with negative count and use previous day count
# think about infering back records instead of setting to day prior 
# add count in new column
states_df2.loc[:,'daily_case_count2'] = states_df2['daily_case_count'].shift(1).where(states_df2['daily_case_count'] <0)
states_df2.loc[:,'daily_deaths_count2'] = states_df2['daily_death_count'].shift(1).where(states_df2['daily_death_count'] <0)

# find negative counts and replace with previous value
states_df2.loc[:,'daily_case_count'] = fix_negative_values(states_df2,'daily_case_count', 'daily_case_count2')
states_df2.loc[:,'daily_death_count'] = fix_negative_values(states_df2,'daily_death_count', 'daily_deaths_count2')

# # add locations to states
states_df2 = pd.merge(states_df2, df_csv, left_on='state', right_on='state_name')

# format to wanted columns
states_df2 = states_df2[['state', 'cases', 'deaths', 'date', 'daily_case_count', 'daily_death_count', 'lat', 'lon']]
states_df2.set_index(pd.DatetimeIndex(states_df2['date']), inplace=True)

In [ ]:
states_df2.tail(10)

# Create Tidy Data DFs

In [ ]:
states_cases_df = states_df2[['state', 'daily_case_count','lat', 'lon']]
states_deaths_df = states_df2[['state', 'daily_death_count','lat', 'lon']]

# time limit df
state_cases_past_xdays = date_limit(states_cases_df,days_back=DAYS_BACK)

# create sample dfs 10% of the data
sample_states_cases = states_cases_df.sample(frac =.10)
sample_states_deaths = states_deaths_df.sample(frac =.10)
sample_state_cases_past_xdays = state_cases_past_xdays.sample(frac =.10)

# dates list
dates_list = sorted(list(set(states_df2.loc[:,'date'].tolist())))

# create most recent date states df
most_recent_date_states_df = states_df2.groupby('state').tail(1)
most_recent_date_cases_df = most_recent_date_states_df[['state','daily_case_count']]
most_recent_date_deaths_df = most_recent_date_states_df[['state','daily_death_count']]

# create past x days cases sum df
state_cases_past_xdays_sum = state_cases_past_xdays.groupby('state')['daily_case_count'].agg('sum').to_frame().reset_index()
state_cases_past_xdays_sum.rename(columns = {'daily_case_count':'total_cnt'}, inplace = True)

# create dates count df
s = states_df2.groupby(states_df2.index)['daily_case_count'].sum()
date_cnts_df = s.to_frame().reset_index()

# need to sort df by state, county, & date to calculate county cases
st_cnty_sorted = df.sort_values(['state','county','date'])

st_cnty_df = st_cnty_sorted.groupby(['state','county']).apply(determine_daily_counts)
st_cnty_df.set_index(pd.DatetimeIndex(st_cnty_df['date']), inplace=True)
st_cnty_df_past_xdays = date_limit(st_cnty_df,days_back=DAYS_BACK)

# create state, county df's
st_cnty_cases_df = st_cnty_df[['state', 'county','fips','daily_case_count']]
st_cnty_cases_df_past_xdays = st_cnty_df_past_xdays[['state', 'county','fips','daily_case_count']]
st_cnty_deaths_df = st_cnty_df[['state', 'county','fips','daily_death_count']]

# time limit states with county df
st_cnty_cases_past_xdays = date_limit(st_cnty_cases_df,days_back=DAYS_BACK)

In [ ]:
states_cases_df.shape

In [ ]:
sample_states_cases.shape

In [ ]:
state_cases_past_xdays.shape

In [ ]:
sample_state_cases_past_xdays.shape

# EDA Section

In [ ]:
# state_cases_past_xdays.info()

In [ ]:
# estimate location of data
determine_est_of_location(sample_state_cases_past_xdays, 'daily_case_count')

In [ ]:
# estimate variability of data
determine_est_of_variability(sample_state_cases_past_xdays, 'daily_case_count')

In [ ]:
# determine whether the data is symmetric or skewed
determine_skewness(sample_state_cases_past_xdays, 'daily_case_count')

In [ ]:
# sample_state_cases_past_xdays['daily_case_count'].quantile([0.05,.25,.5,.75,.95])

In [ ]:
# plot hist graphing distribution
# ax = sample_state_cases_past_xdays.hist(column=['daily_case_count'], bins='auto', alpha=0.5, figsize=(12,8))

In [ ]:
fig = px.histogram(sample_state_cases_past_xdays, x='daily_case_count', histnorm='percent', marginal='rug', nbins=20)
fig.show()

In [ ]:
fig = px.violin(sample_state_cases_past_xdays, y='daily_case_count', points='all', height=600)
fig.show()

# Describe Most Recent Day Cases

In [ ]:
print(f'Date: {most_recent_date_cases_df.index.tolist()[0].strftime("%Y-%m-%d")}')

determine_est_of_location(most_recent_date_cases_df, 'daily_case_count')
determine_est_of_variability(most_recent_date_cases_df, 'daily_case_count')

# determine percentage increase/decrease of current cases compared to previous day
previous_day_cnt = states_df2.loc[states_df2.index == dates_list[-2]]['daily_case_count'].sum()
total_daily_case_count = most_recent_date_states_df['daily_case_count'].sum()
most_rc_date = most_recent_date_states_df['date'].tolist()[0]

# check to see if new total is less than previous day
# if so determine difference
if total_daily_case_count < previous_day_cnt:
    decrease = previous_day_cnt - total_daily_case_count
    percnt_decrease = round(decrease / previous_day_cnt * 100, 2)
    print(f'\n{most_rc_date} had {total_daily_case_count} total cases, ' 
          f'with a decrease of {decrease} or {percnt_decrease}% compared to previous day')
else:
    increase = total_daily_case_count - previous_day_cnt
    percnt_increase = round(increase / previous_day_cnt * 100,2)
    print(f'\n{most_rc_date} had {total_daily_case_count} total cases, ' 
          f'with an increase of {increase} or {percnt_increase}% compared to previous day')

In [ ]:
# top 10 states with most cases for most recent
most_recent_date_cases_df.nlargest(10,'daily_case_count')

In [ ]:
# top 10 states with most cases for past x days
state_cases_past_xdays_sum.nlargest(10,'total_cnt')

# Graph Data

In [ ]:
date_cnts_df.set_index(pd.DatetimeIndex(date_cnts_df['date']), inplace=True)
# plot US daily totals
graph_data(date_cnts_df, 'daily_case_count', 'line', chart_name='Daily Cases', chart_title=f'US {DAYS_BACK} Day Totals',color='royalblue')

In [ ]:
map_data(state_cases_past_xdays, 'daily_case_count', 'US Daily Cases By Date', 'scatter')

In [ ]:
dropdown_state = widgets.Dropdown(options = unique_sorted_values_plus_ALL(states_df2.state), description='State: ')

output_state = widgets.Output()
cases_plot_output = widgets.Output()
deaths_plot_output = widgets.Output()
weekday_plot = widgets.Output()
distro_plot = widgets.Output()
density_plot = widgets.Output()

def event_action():
    # clear the previous selection on each iteration
    output_state.clear_output()
    cases_plot_output.clear_output()
    deaths_plot_output.clear_output()
    weekday_plot.clear_output()
    distro_plot.clear_output()
    density_plot.clear_output()
    
    if (dropdown_state.value == ALL):
        common_filter = states_df2
        common_map_filter = st_cnty_df
    else:
        common_filter = states_df2[states_df2.state == dropdown_state.value]
        common_map_filter = st_cnty_df_past_xdays[st_cnty_df_past_xdays.state == dropdown_state.value]
    
    with output_state:
        display(common_filter.last(pd.offsets.Day(DAYS_BACK)))
    
    with cases_plot_output:
        graph_data(common_filter, 'daily_case_count', 'line', chart_name='Daily Cases', chart_title=common_filter.state[0],color='royalblue')
        
    with deaths_plot_output:
        graph_data(common_filter, 'daily_death_count', 'line',chart_name='Daily Cases', chart_title=common_filter.state[0], color='firebrick')
    
    with weekday_plot:
        graph_data(common_filter, 'daily_case_count', 'bar',)
    
    with distro_plot:
        graph_data(common_filter, 'daily_case_count', 'histogram')
    
    with density_plot:
        
        # group data by county and get daily case sum
        common_map_filter = common_map_filter.groupby(['county', 'fips'], as_index=False)['daily_case_count'].agg('sum')
        common_map_filter.rename(columns = {'daily_case_count':'total_cnt'}, inplace = True)
        print(f'Top 10 counties with most cases in past {DAYS_BACK} days:\n {common_map_filter.nlargest(10,"total_cnt")}')
        
        map_data(common_map_filter,'total_cnt', 'State Cases Density', 'choropleth')

def dropdown_state_eventhandler(change):
    event_action()
    
def graphit():
    event_action()
    
def tab_chg(chg):
    if chg.old == {}:
        graphit()

dropdown_state.observe(dropdown_state_eventhandler, names='value')

In [ ]:
input_widgets = widgets.HBox([dropdown_state])

tab = widgets.Tab([output_state, distro_plot, cases_plot_output, deaths_plot_output, weekday_plot, density_plot])
tab.set_title(0, 'Dataset')
tab.set_title(1, 'Distribution Graph')
tab.set_title(2, 'Daily Cases Graph')
tab.set_title(3, 'Daily Deaths Graph')
tab.set_title(4, 'Weekday Counts')
tab.set_title(5, 'State Density Plot')
tab.observe(tab_chg)

dashboard = widgets.VBox([tab], layout=Layout(height='700px'))
display(input_widgets, dashboard)

In [ ]:
# data used for filter operations
df3 = st_cnty_df

# create initial drop down options
# countyW = widgets.Dropdown(options = df3.county.unique(), description='County: ')

def filter_and_graph(**func_kwargs):
    """
    filters df and calls graph function
    based on selected operations
    """
    
    # convert user params to list
    l = list(func_kwargs.values())
    # filter df based on updated values
    df_updated = df3.query(f'state=="{l[0]}" and county=="{l[1]}"')
    
    # call function to graph data
    graph_data(df_updated, 'daily_case_count', 'line', chart_name='Daily Cases', chart_title=l[1],color='royalblue')
    
def select_country(state):
    """
    helper function to handle
    state changes and update
    """
    
    # update widgets
    new_i = widgets.interactive(filter_and_graph, state=stateW, county=df3.query(f'state=="{state["new"]}"').county.unique())
    i.children = new_i.children

# initial dropdown options
stateW = widgets.Dropdown(options=df3.state.unique(), description='state: ')
init = stateW.value
countyW = df3.query(f'state=="{init}"').county.unique()

# observe selection changes from dropdown and caller helper function
# to update second dropdown list
stateW.observe(select_country, 'value')

i = widgets.interactive(filter_and_graph, state=stateW, county=countyW)

display(i)